# Adaptation prompt
It's a prefix prompt tunning method.

In [1]:
import os
from enum import Enum
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from peft import get_peft_config, get_peft_model, AdaptionPromptConfig, TaskType, PeftType
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

Make sure access to Llama2 has been granted in huggingface

In [2]:
os.environ["WANDB_PROJECT"] = "prompt_learning_methods"
seed = 0
set_seed(seed)
device="cuda"
model_name_or_path="meta-llama/Llama-2-7b-hf"
tokenizer_name_or_path = "meta-llama/Llama-2-7b-hf"

## Load dataset

In [3]:
dataset_name = "jaykin01/advertisement-copy"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
template = """{% for message in messages %}\n{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% if loop.last and add_generation_prompt %}{{'<|im_start|>assistant\n' }}{% endif %}{% endfor %}"""
tokenizer.chat_template = template
system_prompt = """Create a text ad given the following product and description."""

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
def preprocess(samples):
    batch = []
    for product, desc, ad_copy in zip(samples["product"], samples["description"], samples["ad"]):
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""Product: {product}\nDescription: {desc}\n"""},
            {"role": "assistant", "content": f"""Ad: {ad_copy}\n"""}
        ]
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    
    return {"content": batch}

In [5]:
dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)
# see example
dataset["train"][0]

Generating train split:   0%|          | 0/1141 [00:00<?, ? examples/s]

Map:   0%|          | 0/1141 [00:00<?, ? examples/s]

{'content': '<|im_start|>system\nCreate a text ad given the following product and description.<|im_end|>\n<|im_start|>user\nProduct:  Harem pants\nDescription:  A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.\n<|im_end|>\n<|im_start|>assistant\nAd: Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!\n<|im_end|>\n'}

In [10]:
# split data
dataset = dataset["train"].train_test_split(0.1)

## Create PEFT model
### Adaptation Prompt Tuning config

In [11]:
peft_config = AdaptionPromptConfig(
    adapter_len=32,
    adapter_layers=30,
    task_type=TaskType.CAUSAL_LM
)

In [12]:
class ChatmlSpecialTokens(str, Enum):
    user = "<|im_start|>user"
    assistant = "<|im_start|>assistant"
    system = "<|im_start|>system"
    eos_token = "<|im_end|>"
    bos_token = "<s>"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]

response_template = "<|im_start|>assistant\n"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    pad_token=ChatmlSpecialTokens.pad_token.value,
    bos_token=ChatmlSpecialTokens.bos_token.value,
    eos_token=ChatmlSpecialTokens.eos_token.value,
    additional_special_tokens=ChatmlSpecialTokens.list(),
    trust_remote_code=True
)
tokenizer.chat_template = template
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [13]:
# create model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# cast non-trainable params in fp16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 3,932,190 || all params: 6,742,388,766 || trainable%: 0.0583


## Training

In [14]:
# define hyperparams and arguments

output_dir = "llama_adpcopy"
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
gradient_accumulation_steps = 1
logging_steps = 5
learning_rate = 5e-4
max_grad_norm = 1.0
max_steps = 250
num_train_epochs = 10
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 512

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=True,
    report_to=["tensorboard", "wandb"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    packing=False,
    dataset_text_field="content",
    max_seq_length=max_seq_length,
    data_collator=collator
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/923 [00:00<?, ? examples/s]

Map:   0%|          | 0/103 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer.train()
trainer.save_model()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: jdgallegoq (jdgallegoq-wandb). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss
1,1.576300,1.581163
2,0.999300,0.948806
3,0.804000,0.821333
4,0.709300,0.775385
5,0.679500,0.750503
6,0.698300,0.737763
7,0.633700,0.726564
8,0.583000,0.724837
9,0.681600,0.722086
10,0.702500,0.722270


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

events.out.tfevents.1724176382.76fab6e09ec0:   0%|          | 0.00/57.1k [00:00<?, ?B/s]

## Load for inference

In [17]:
from peft import PeftModel, PeftConfig

In [19]:
peft_model_id = "jdgallegoq96/llama_adpcopy"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/488 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [21]:
# inference
model.to(torch.float16)
model.cuda()
model.eval()
messages = [
    {"role": "system", "content": "Create a text ad given the following product and description."},
    {"role": "user", "content": "Product: Sony PS5 PlayStation Console\nDescription: The PS5™ console unleashes new gaming possibilities that you never anticipated."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
otputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    top_p=0.95,
    temperature=0.2,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(otputs[0]))

<s><|im_start|>system 
Create a text ad given the following product and description.<|im_end|> 
<|im_start|>user 
Product: Sony PS5 PlayStation Console
Description: The PS5™ console unleashes new gaming possibilities that you never anticipated.<|im_end|> 
<|im_start|>assistant 
Ad: Unlock immersive gaming, experience next-gen thrills! 🎮🌟 Enjoy stunning graphics and gameplay flexibility. Perfect for gamers and embracing the future of gaming with a touch of PS5™ magic! 🌟🕹️📱
<|im_end|>
